#### Trick-or-treat.ipynb: going house to house in each neighborhood

In [167]:
import pandas as pd
import os
from lxml import html
import requests
import time

In [138]:
df = pd.read_csv('./data/texas/austin-central-texas.csv')

In [139]:
df.shape

(232, 8)

In [140]:
def cleanURL(url):
    """
    Takes lennar url as in all the csv files and returns the updated version ready to scrape
    returns: String (url)
    """
    
    ## get rid of code if the last part of url starts with a digit
    if url.split('/')[-1][0].isdigit():
        tmp = url.split('/')[:-1]
        url = '/'.join(tmp)
    
    final_url = url + '/included?print=true'
    return final_url

In [141]:
csv_files = []
path = './data/'
for root, dirs, files in os.walk(path):
    for name in files:
        if name.endswith('.csv'):
            csv_files.append(root+'/'+name)

In [142]:
csv_files[:1]

['./data/north-carolina/charlotte.csv']

In [170]:
## main loop

for f in csv_files[:1]:
    df = pd.read_csv(f)
    
    data_dump = [] # list which will be one column in the new df
    new_rows = [] # list of lists representing rows for each home of different length with each attribute
    
    for u in df['URL']:
        newURL = cleanURL(u)
        time.sleep(2)
        print(newURL)
        
        try:
            page = requests.get(newURL)
            tree = html.fromstring(page.content)  
            index = 3
            keep_scraping = True
            d = dict()
            
            if page.status_code == 200 and 'redirectNotice=true' in page.url:
                data_dump.append(d)
                new_rows.append([])
                continue
            
            else:
                while keep_scraping:
                    try:
                        key = tree.xpath(f'//*[@id="main"]/div/div/div/div[{index}]/div[1]/text()')[0]
                        val = tree.xpath(f'//*[@id="main"]/div/div/div/div[{index}]/div[2]/*/text()')

                        index += 1
                        d[key] = val
                    except IndexError:
                        del d[key] # remove the last key we looked at which is the extra info at bottom of page
                        # print(d)
                        keep_scraping = False

                data_dump.append(d)
                new_rows.append([item for sublist in d.values() for item in sublist])
                # print(data_dump)
                # print(new_rows)
                
        except: 
            print("Something is wrong with connecting to the webpage -- maybe with URL")
            break
    
    ## add collected dataframe back
    
    ## resave it
        

https://www.lennar.com/new-homes/south-carolina/charlotte/tega-cay/windhaven/meadows/dover-basement/included?print=true
https://www.lennar.com/new-homes/north-carolina/charlotte/mint-hill/estates-at-arlington-woods/dover/included?print=true
https://www.lennar.com/new-homes/north-carolina/charlotte/mooresville/gambill-forest/enclave/davidson/included?print=true
https://www.lennar.com/new-homes/north-carolina/charlotte/huntersville/northbrook/signature/ellery/included?print=true
https://www.lennar.com/new-homes/north-carolina/charlotte/mint-hill/estates-at-arlington-woods/grisham/included?print=true
https://www.lennar.com/new-homes/south-carolina/charlotte/lancaster/walnut-creek/walk/sweetbay/included?print=true
https://www.lennar.com/new-homes/north-carolina/charlotte/mooresville/gambill-forest/enclave/durham/included?print=true
https://www.lennar.com/new-homes/north-carolina/charlotte/mooresville/gambill-forest/enclave/camden/included?print=true
https://www.lennar.com/new-homes/north-c

In [172]:
print(len(data_dump))
print(len(new_rows))

199
199


In [165]:
df.head()

,Availability,Price,Beds,Baths,Sqft,Address,Community,URL
0,Move-in ready,"$496,999",3.0,2.5,4164.0,"3171 Windhaven Lane, Tega Cay, SC",Dover Basement in Windhaven,https://www.lennar.com/new-homes/south-carolin...
1,Move-in ready,"$540,959",3.0,2.5,2082.0,"11112 Dappled Light Trail, Mint Hill, NC",Dover in Estates at Arlington Woods,https://www.lennar.com/new-homes/north-carolin...
2,Move-in ready,"$458,679",5.0,3.5,3257.0,"122 Doncaster Drive, Mooresville, NC",Davidson in Gambill Forest,https://www.lennar.com/new-homes/north-carolin...
3,Move-in ready,"$661,839",5.0,4.0,3084.0,"19108 Bird Meadow Drive, Huntersville, NC",Ellery in Northbrook,https://www.lennar.com/new-homes/north-carolin...
4,Move-in ready,"$603,619",5.0,4.5,3269.0,"11022 Dappled Light Trail, Mint Hill, NC",Grisham in Estates at Arlington Woods,https://www.lennar.com/new-homes/north-carolin...


In [56]:
mylist = []
mylist.append(df['URL'][1])
mylist.append(df['URL'][230])

In [69]:
for url in mylist:
    newURL = cleanURL(url)
    print(newURL)

https://www.lennar.com/new-homes/texas/austin-central-texas/georgetown/wolf-ranch/claremont-collection/ames/included?print=true
https://www.lennar.com/new-homes/texas/austin-central-texas/san-marcos/whisper/highlands-collections/pierson/included?print=true


In [128]:
### key:
### val: //*[@id="main"]/div/div/div/div[4]/div[2] -- each line

try:
    page = requests.get(newURL)
    tree = html.fromstring(page.content)  
    index = 3
    keep_scraping = True
    d = dict()
    
    while keep_scraping:
        try:
            key = tree.xpath(f'//*[@id="main"]/div/div/div/div[{index}]/div[1]/text()')[0]
            val = tree.xpath(f'//*[@id="main"]/div/div/div/div[{index}]/div[2]/*/text()')
        
            index += 1
            d[key] = val
        except IndexError:
            del d[key] # remove the last key we looked at which is the extra info at bottom of page
            print(d)
            keep_scraping = False

except: 
    print("Something is wrong with connecting to the webpage -- maybe with URL")

{'Living Room': ['Dual pane low-E windows with designer blinds'], 'Kitchen': ['Brand-new stainless steel appliance package', 'GE® stainless steel multicycle dishwasher', 'GE® stainless steel slide-in gas range', 'GE® stainless steel microwave', 'Stainless steel undermount sink', 'Moen® faucet', 'Quartz countertops', '3"x6" tile backsplash', 'Designer-selected cabinetry'], 'Owners Suite': ['Moen® chrome faucets', 'Solid surface countertops', 'Shower and bathtub combination with tile surround', 'Moen® bath fixtures and faucets'], 'Secondary Bathroom': ['Tile flooring in full-sized bathrooms on 2nd Floor (Per Plan)', 'Solid surface countertops', 'Moen® bath fixtures and faucets'], 'Interior': ['Shaw® luxury vinyl plank flooring', 'Plus wall-to-wall carpeting in designated areas', 'Tile floors in designated areas'], 'Exterior': ['Insulated fiberglass six-panel front door', 'Full sod and irrigation', 'PEX plumbing system', 'Insulated garage doors', 'Sherwin-Williams® exterior paint', 'Fence

In [136]:
elems = [item for sublist in d.values() for item in sublist]
elems

['Dual pane low-E windows with designer blinds',
 'Brand-new stainless steel appliance package',
 'GE® stainless steel multicycle dishwasher',
 'GE® stainless steel slide-in gas range',
 'GE® stainless steel microwave',
 'Stainless steel undermount sink',
 'Moen® faucet',
 'Quartz countertops',
 '3"x6" tile backsplash',
 'Designer-selected cabinetry',
 'Moen® chrome faucets',
 'Solid surface countertops',
 'Shower and bathtub combination with tile surround',
 'Moen® bath fixtures and faucets',
 'Tile flooring in full-sized bathrooms on 2nd Floor (Per Plan)',
 'Solid surface countertops',
 'Moen® bath fixtures and faucets',
 'Shaw® luxury vinyl plank flooring',
 'Plus wall-to-wall carpeting in designated areas',
 'Tile floors in designated areas',
 'Insulated fiberglass six-panel front door',
 'Full sod and irrigation',
 'PEX plumbing system',
 'Insulated garage doors',
 'Sherwin-Williams® exterior paint',
 'Fence in rear yard for privacy',
 'Professionally-landscaped front yard',
 'eer

In [132]:
# each row is a house
# each house has basic info
# each column -- feature 1, feature 2, feature 3
# dump dictionary